# Linear Classification

In this lab you will implement parts of a linear classification model using the regularized empirical risk minimization principle. By completing this lab and analysing the code, you gain deeper understanding of these type of models, and of gradient descent.


## Problem Setting

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal (1) and abnormal (0). The training data contains 80 SPECT images from which 22 binary features have been extracted. The goal is to predict the label for an unseen test set of 187 tomography images.

In [1]:
import urllib.request
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

testfile = urllib.request.URLopener()
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train", "SPECT.train")
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test", "SPECT.test")

df_train = pd.read_csv('SPECT.train',header=None)
df_test = pd.read_csv('SPECT.test',header=None)

train = df_train.values
test = df_test.values

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]

### Exercise 1

Analyze the function learn_reg_ERM(X,y,lambda) which for a given $n\times m$ data matrix $\textbf{X}$ and binary class label $\textbf{y}$ learns and returns a linear model $\textbf{w}$.
The binary class label has to be transformed so that its range is $\left \{-1,1 \right \}$.
The trade-off parameter between the empirical loss and the regularizer is given by $\lambda > 0$.
To adapt the learning rate the Barzilai-Borwein method is used.

Try to understand each step of the learning algorithm and comment each line.


In [ ]:
def learn_reg_ERM(X,y,lbda):
    max_iter = 200
    e  = 0.001
    alpha = 1.

    w = np.random.randn(X.shape[1]);
    for k in np.arange(max_iter):
        h = np.dot(X,w)
        l,lg = loss(h, y)
        print ('loss: {}'.format(np.mean(l)))
        r,rg = reg(w, lbda)
        g = np.dot(X.T,lg) + rg
        if (k > 0):
            alpha = alpha * (np.dot(g_old.T,g_old))/(np.dot((g_old - g).T,g_old))
        w = w - alpha * g
        if (np.linalg.norm(alpha * g) < e):
            break
        g_old = g
    return w

In [2]:
def learn_reg_ERM(X, y, lbda):
    # Set the maximum number of iterations for the optimization loop
    max_iter = 200

    # Convergence threshold: stop if the weight update is smaller than this value
    e = 0.001

    # Initialize the learning rate
    alpha = 1.

    # Randomly initialize the weights (model parameters), one for each feature
    w = np.random.randn(X.shape[1])

    # Start the iterative optimization process
    for k in np.arange(max_iter):
        # Compute the raw predictions (linear combination of features and weights)
        h = np.dot(X, w)

        # Compute the loss and its gradient with respect to h
        l, lg = loss(h, y)

        # Print the average loss for monitoring progress
        print('loss: {}'.format(np.mean(l)))

        # Compute the regularization term and its gradient with respect to weights
        r, rg = reg(w, lbda)

        # Compute the total gradient: gradient of data loss + gradient of regularization
        g = np.dot(X.T, lg) + rg

        # Adapt the learning rate using Barzilai-Borwein method (only after the first iteration)
        if (k > 0):
            alpha = alpha * (np.dot(g_old.T, g_old)) / (np.dot((g_old - g).T, g_old))

        # Update the weights using the gradient descent step
        w = w - alpha * g

        # Stop if the weight update is small enough (convergence criterion)
        if (np.linalg.norm(alpha * g) < e):
            break

        # Store the current gradient for use in the next iteration
        g_old = g

    # Return the optimized weights
    return w


### Exercise 2

Fill in the code for the function loss(h,y) which computes the hinge loss and its gradient.
This function takes a given vector $\textbf{y}$ with the true labels $\in \left \{-1,1\right \}$ and a vector $\textbf{h}$ with the function values of the linear model as inputs. The function returns a vector $\textbf{l}$ with the hinge loss $\max(0, 1 − y_{i} h_{i})$ and a vector $\textbf{g}$ with the gradients of the hinge loss w.r.t $\textbf{h}$. (Note: The partial derivative of the hinge loss with respect to $\textbf{h}$  is $g_{i} = −y $ if $l_{i} > 0$, else $g_{i} = 0$)

In [5]:
def loss(h, y):
    # Compute the hinge loss for each sample
    l = np.maximum(0, 1 - y * h)
    print(f'value of l: {l}')
    # Initialize gradient vector with zeros
    g = np.zeros_like(y)
    print(f'value of g: {g}')

    # Compute gradient: only non-zero where loss > 0
    g[l > 0] = -y[l > 0]

    return l, g

# h = np.array([2.0, -0.5, 0.8])
# y = np.array([1, -1, 1])
# loss(h,y)

value of l: [0.  0.5 0.2]
value of g: [0 0 0]


(array([0. , 0.5, 0.2]), array([ 0,  1, -1]))

### Exercise 3

Fill in the code for the function reg(w,lambda) which computes the $\mathcal{L}_2$-regularizer and the gradient of the regularizer function at point $\textbf{w}$.


$$r = \frac{\lambda}{2} \textbf{w}^{T}\textbf{w}$$

$$g = \lambda \textbf{w}$$

In [8]:
def reg(w, lbda):
    # Compute the L2 regularization loss:
    # (lambda / 2) * sum(w_i^2)
    r = 0.5 * lbda * np.dot(w, w)

    # Compute the gradient of the regularization term with respect to weights:
    # lambda * w
    g = lbda * w

    return r, g

# w = np.array([1.0, 2.0, 3.0])
# lbda = 0.1

# reg(w,lbda)

(np.float64(0.7000000000000001), array([0.1, 0.2, 0.3]))

### Exercise 4

Fill in the code for the function predict(w,x) which predicts the class label $y$ for a data point $\textbf{x}$ or a matrix $X$ of data points (row-wise) for a previously trained linear model $\textbf{w}$. If there is only a data point given, the function is supposed to return a scalar value. If a matrix is given a vector of predictions is supposed to be returned.

In [10]:
def predict(w, X):
    # If X is a 1D array (single data point), reshape it to a 2D row vector
    if X.ndim == 1:
        h = np.dot(w, X)  # scalar prediction
        preds = 1 if h > 0 else -1  # sign function
    else:
        # Compute raw scores for each row in the matrix
        h = np.dot(X, w)  # vector of predictions
        preds = np.where(h > 0, 1, -1)  # vectorized sign function

    return preds

# w = np.array([0.2, -0.5])
# X_point = np.array([1.0, 2.0])
# X_matrix = np.array([[1.0, 2.0], [-1.0, -2.0]])

# print(predict(w, X_point))   # Output: -1 (scalar)
# print(predict(w, X_matrix))  # Output: [-1  1] (vector)


-1
[-1  1]


### Exercise 5

#### 5.1
Train a linear model on the training data and classify all 187 test instances afterwards using the function predict.
Please note that the given class labels are in the range $\left \{0,1 \right \}$, however the learning algorithm expects a label in the range of $\left \{-1,1 \right \}$. Then, compute the accuracy of your trained linear model on both the training and the test data.

In [11]:
# Step 1: Convert class labels from {0,1} to {-1,1}
y_train_bin = np.where(y_train == 0, -1, 1)
y_test_bin = np.where(y_test == 0, -1, 1)

# Step 2: Train the model with regularization parameter lambda (e.g., 0.1)
lambda_val = 0.1
w_trained = learn_reg_ERM(X_train, y_train_bin, lambda_val)

# Step 3: Predict on training and test data
train_preds = predict(w_trained, X_train)
test_preds = predict(w_trained, X_test)

# Step 4: Convert predictions back to {0,1} to match original labels
train_preds_01 = np.where(train_preds == -1, 0, 1)
test_preds_01 = np.where(test_preds == -1, 0, 1)

# Step 5: Compute accuracy
train_accuracy = np.mean(train_preds_01 == y_train)
test_accuracy = np.mean(test_preds_01 == y_test)

print("Training Accuracy: {:.2f}%".format(train_accuracy * 100))
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))


value of l: [0.         0.15447577 0.         4.59192772 0.12781258 2.06483946
 0.         2.51515344 0.24542333 2.15097511 0.         0.
 0.         2.12555731 0.         1.36751435 0.         2.2481193
 0.         0.         2.96766491 0.         0.         1.27580246
 0.         1.79628815 0.         2.88661085 2.43072536 0.
 1.         1.         0.         0.26790105 0.         0.
 0.         0.         0.         0.         0.         1.21287959
 2.97874081 1.38193686 1.19061203 3.98508344 0.         1.08462919
 1.         0.         1.77762807 1.         1.         1.40425398
 3.5098558  3.5680123  0.25923855 7.87781141 1.         1.
 4.2144619  1.         0.         1.         1.         1.
 0.         1.         5.10712704 1.         0.70269233 2.62758113
 6.08643057 4.66625576 1.         5.4158088  9.17403445 6.5616582
 0.23307527 5.10712704]
value of g: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

#### 5.2
Compare the accuracy of the linear model with the accuracy of a random forest and a decision tree on the training and test data set.

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
tree_train_preds = tree_clf.predict(X_train)
tree_test_preds = tree_clf.predict(X_test)

# Train random forest
rf_clf = RandomForestClassifier(random_state=42, n_estimators=100)
rf_clf.fit(X_train, y_train)
rf_train_preds = rf_clf.predict(X_train)
rf_test_preds = rf_clf.predict(X_test)

# Compute accuracies
print("Accuracy Comparison:")
print("Linear Model - Train: {:.2f}%, Test: {:.2f}%".format(
    accuracy_score(y_train, train_preds_01) * 100,
    accuracy_score(y_test, test_preds_01) * 100
))
print("Decision Tree - Train: {:.2f}%, Test: {:.2f}%".format(
    accuracy_score(y_train, tree_train_preds) * 100,
    accuracy_score(y_test, tree_test_preds) * 100
))
print("Random Forest - Train: {:.2f}%, Test: {:.2f}%".format(
    accuracy_score(y_train, rf_train_preds) * 100,
    accuracy_score(y_test, rf_test_preds) * 100
))

Accuracy Comparison:
Linear Model - Train: 85.00%, Test: 72.73%
Decision Tree - Train: 93.75%, Test: 67.91%
Random Forest - Train: 93.75%, Test: 76.47%
